In [ ]:
# Basic Packages
import numpy as np
import h5py
import logging
import os
import shutil
import gc
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import linregress, pearsonr, spearmanr
import seaborn as sns
from scipy.optimize import curve_fit
import scipy.stats as stats
from scipy.ndimage import gaussian_filter
import matplotlib.patches as patches
from skimage.feature import peak_local_max
from matplotlib.collections import PatchCollection
from scipy.stats import pearsonr
from astropy.io import fits
from scipy.stats import kendalltau

# Physics-related Packages
from astropy.cosmology import Planck15
from astropy.wcs import WCS

In [ ]:
SN_Path = '/users_path/merger_trace/data/tng300/tng300_aperture_mass_maps'
cat_name = '/home/chuiyang/merger_trace/data/tng300/tng300_targetcat/TargetHalo_MergerCat_092.hdf5'

In [ ]:
# preparations for read box info from the url
import requests

baseUrl = 'http://www.tng-project.org/api/'
headers = {"api-key":"API KEY"}

def get(path, params=None):
    # make HTTP GET request to path
    r = requests.get(path, params=params, headers=headers)

    # raise exception if response code is not HTTP SUCCESS (200)
    r.raise_for_status()

    if r.headers['content-type'] == 'application/json':
        return r.json() # parse json responses automatically

    if 'content-disposition' in r.headers:
        filename = r.headers['content-disposition'].split("filename=")[1]
        with open(filename, 'wb') as f:
            f.write(r.content)
        return filename # return the filename string

    return r

In [ ]:
# Issue a request to the API root
r = get(baseUrl)

# Print out all the simulation names
names = [sim['name'] for sim in r['simulations']]
# Get the index of TNG300-1
i = names.index('TNG300-1')
# Get the info of simulation Illustris-3
sim = get( r['simulations'][i]['url'] )
sim.keys()

# get the snaps info this simulation
snaps = get(sim['snapshots'])

In [ ]:
# Sim Box parameters
Snap_Index = 92 # the snapshots index in the total 100 snapshots taking at different z
BoxSize = sim['boxsize'] # unit: ckpc/h
Redshift = snaps[Snap_Index]['redshift'] # current redshift of our current snap

# Short description of cosmological parameters Planck2015
h_atz = Planck15.H(Redshift).value/100 # unit 100km/[s*Mpc]

LowerMass_lim = 5*10**3 * h_atz # units 10^ 10 𝑀⊙/ℎ * h

In [ ]:
def find_peaks(SN_map, min_distance, threshold_abs):
    # Extract peak positions
    peaks = peak_local_max(SN_map, min_distance=min_distance, threshold_abs=threshold_abs)
    # Sort the peaks by intensity (highest to lowest)
    peaks_sorted = np.array(sorted(peaks, key=lambda p: SN_map[p[0], p[1]], reverse=True))

    return peaks_sorted

In [ ]:
def get_mass_ratio(cat_name, halo_id):
    with h5py.File(cat_name) as cat:
        Halo_IDs = cat['Group/FOF_Halo_IDs'][:]
        GroupPos = cat['Group/GroupPos'][:]
        Group_R_Crit200 = cat['Group/Group_R_Crit200'][:]
        
        SubhaloGrNr = cat['Subhalo/SubhaloGrNr'][:]
        SubhaloMasses = cat['Subhalo/SubhaloMass'][:]
        SubhaloPos = cat['Subhalo/SubhaloPos'][:]

    halo_pos =  GroupPos[Halo_IDs == halo_id][0]
    halo_x, halo_y, halo_z = halo_pos[0], halo_pos[1], halo_pos[2]
    halo_r = Group_R_Crit200[Halo_IDs==halo_id]

    Subhalo_in = np.where(
        (SubhaloGrNr == halo_id) &
        (SubhaloPos[:,0] <= halo_x + halo_r) &
        (SubhaloPos[:,0] >= halo_x - halo_r) &
        (SubhaloPos[:,1] <= halo_y + halo_r) &
        (SubhaloPos[:,1] >= halo_y - halo_r) &
        (SubhaloPos[:,2] <= halo_z + 2*halo_r) &
        (SubhaloPos[:,2] >= halo_z - 2*halo_r) 
    )[0]

    SubhaloMass_in = SubhaloMasses[Subhalo_in]
    sorted_masses = np.sort(SubhaloMass_in)[::-1] 

    # get the most and second massive halos
    if len(sorted_masses) >= 2:
        m1, m2 = sorted_masses[0], sorted_masses[1]
        top_sorted = np.argsort(SubhaloMass_in)[::-1]
        top_indices = Subhalo_in[top_sorted[:2]] 
    elif len(sorted_masses) == 1:
        m1, m2 = sorted_masses[0], 0
        top_indices = [Subhalo_in[0]]
    else:
        m1, m2 = 0, 0
        top_indices = []
    
    # get positions
    top_positions = SubhaloPos[top_indices]  
    
    # convert to pixel pos
    relative_xy = top_positions[:, :2] - halo_pos[:2]
    pixel_coords = (relative_xy / (2*halo_r) * 240).astype(int) + 120  # center shift


    return m1, m2, m2 / m1 if m1 > 0 else np.nan, pixel_coords

In [ ]:
with h5py.File(cat_name) as cat:
    FOF_Halo_IDs = cat['Group/FOF_Halo_IDs'][:]
    Group_M_Crit200 = cat['Group/Group_M_Crit200'][:]

Target_Halo_IDs = FOF_Halo_IDs[Group_M_Crit200>=LowerMass_lim].astype(int)

In [ ]:
def peak_intensity_separation(halo_id, cat_name, sigma=10, min_distance=2, threshold_abs=0.05, if_plot=False):
    # get mass ratio
    mass1, mass2, mass_ratio, pixel_coords =get_mass_ratio(cat_name, halo_id)

    # get sn_map
    file_full = SN_Path+ f'sn_map_full_halo{halo_id}.fits'
    file_nonoise = SN_Path +f'sn_map_nonoise_halo{halo_id}.fits'

    # open fits
    with fits.open(file_full) as hdul:
        SN_map_full = hdul[0].data

    with fits.open(file_nonoise) as hdul:
        SN_map_nonoise = hdul[0].data

    # smooth sn map with noise and find peaks
    smoothed_SN_map = gaussian_filter(SN_map_full, sigma=sigma)
    peaks = find_peaks(smoothed_SN_map, min_distance=min_distance, threshold_abs=threshold_abs)

    if if_plot:
        # plot smoothed sn map, and the 1st 2nd peaks
        fig, axs = plt.subplots(1, 2, figsize=(10, 5))
        im0 = axs[0].imshow(smoothed_SN_map, origin='lower', cmap='viridis')
        if len(peaks) >= 1:
            axs[0].scatter(peaks[0][1], peaks[0][0], s=50, color='red', label='1st Peak', marker='x')
        if len(peaks) >= 2:
            axs[0].scatter(peaks[1][1], peaks[1][0], s=50, color='orange', label='2nd Peak', marker='x')

        axs[0].set_title(f'Smoothed SN Map with Peaks (halo {halo_id})')
        plt.colorbar(im0, ax=axs[0], fraction=0.046, pad=0.04, label='SN Value')


        # no-noise maps and true peak positions (1st 2nd subhalos)
        im1 = axs[1].imshow(SN_map_nonoise, origin='lower', cmap='viridis')
        axs[1].set_title(f'Nonoise SN Map with True Subhalos (halo {halo_id})')
        plt.colorbar(im1, ax=axs[1], fraction=0.046, pad=0.04, label='SN Value')
        
        if len(pixel_coords) >= 1:
            axs[1].scatter(pixel_coords[0][1], pixel_coords[0][0],
                           s=100, color='cyan', marker='*', label='Most Massive')
        
        if len(pixel_coords) == 2:
            axs[1].scatter(pixel_coords[1][1], pixel_coords[1][0],
                           s=100, color='magenta', marker='*', label='2nd Massive')
        
        axs[1].legend()
        plt.tight_layout()
        plt.show()

    if len(peaks)>1:
        pos1 = peaks[0]
        pos2 = peaks[1]
        intensity_1 = smoothed_SN_map[pos1[0], pos1[1]]
        intensity_2 = smoothed_SN_map[pos2[0], pos2[1]]
        separation = np.linalg.norm(np.array(pos1) - np.array(pos2))
    else:
        intensity_1 = -1
        intensity_2 = -1
        separation = -1

    return intensity_1, intensity_2, separation, mass_ratio
        

In [ ]:

with h5py.File(cat_name) as f:
    Group_M_Crit200 = f['Group/Group_M_Crit200'][:]
    FOF_Halo_IDs = f['Group/FOF_Halo_IDs'][:]

'''
mass_ratios = np.zeros(len(FOF_Halo_IDs))
for i, halo_id in enumerate(FOF_Halo_IDs):
    m1, m2, mass_ratio, pixel_coords = get_mass_ratio(cat_name, halo_id)
    print(mass_ratio)
    mass_ratios[i]= mass_ratio
    print(f'finish {i}')
'''

In [ ]:
n = len(Target_Halo_IDs)
intensity_1_vec = np.full(n, np.nan)
intensity_2_vec = np.full(n, np.nan)
separation_vec  = np.full(n, np.nan)
true_mass_ratio_vec = np.full(n, np.nan)

failed_ids = []

for i, halo_id in enumerate(Target_Halo_IDs):
    try:
        intensity_1, intensity_2, separation, true_mass_ratio = peak_intensity_separation(
            halo_id, cat_name, sigma=10, min_distance=2, threshold_abs=0.05, if_plot=False
        )
        intensity_1_vec[i] = intensity_1
        intensity_2_vec[i] = intensity_2
        separation_vec[i]  = separation
        true_mass_ratio_vec[i] = true_mass_ratio

        print(f'finish {halo_id}')
    except Exception as e:
        failed_ids.append((halo_id, str(e)))

        print(f"[skip] halo {halo_id}: {e}")


In [ ]:
# create dataframe
df = pd.DataFrame({
    'intensity_1': intensity_1_vec,
    'intensity_2': intensity_2_vec,
    'separation': separation_vec,
    'true_mass_ratio': true_mass_ratio_vec
})

# save to .csv
df.to_csv('halo_peak_separation.csv', index=False)
